In [1]:
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
from py_vollib.black_scholes import black_scholes

In [2]:
data=pd.read_csv('INPUT/EICHERMOT.csv')
data['IV Drop']=(-data['post_announcement_ivs_open']+data['pre_announcement_ivs'])*100
data['pre_iv']=data['pre_announcement_ivs']*100
data['Price_change']=data['Close_open_spot_move']
data=data[['Announcement Date','IV Drop','Price_change','pre_iv']]
data.index=pd.to_datetime(data['Announcement Date'])
#data.dropna(inplace=True)
current_call_iv=0.26
current_put_iv=0.31
strike_ce=3400
strike_pe=3400
fut=3400
lot_size=175

In [3]:
pre_prem=[]
post_prem=[]
for i in data.index:
    print(i,"new price",(1+data['Price_change'].loc[i])*fut,data['Price_change'].loc[i],data['pre_iv'].loc[i]/100)
    ce=black_scholes('c',fut,strike_ce,15/365, 0, data['pre_iv'].loc[i]/100)
    pe=black_scholes('p',fut,strike_pe,15/365, 0, data['pre_iv'].loc[i]/100)
    print(ce,pe)
    total_before=ce+pe
    pre_prem.append(total_before)
    ce_post=black_scholes('c',(1+data['Price_change'].loc[i])*fut,strike_ce,14/365, 0, (data['pre_iv'].loc[i]-data['IV Drop'].loc[i])/100)
    pe_post=black_scholes('p',(1+data['Price_change'].loc[i])*fut,strike_pe,14/365, 0, (data['pre_iv'].loc[i]-data['IV Drop'].loc[i])/100)
    total=ce_post+pe_post
    print(ce_post,pe_post,(data['pre_iv'].loc[i]-data['IV Drop'].loc[i])/100)
    print(total)
    post_prem.append(total)
    

2022-08-10 00:00:00 new price 3453.006921953388 0.0155902711627611 0.3692026777201302
101.4966396200827 101.4966396200827
113.63174566502003 60.62482371163206 0.3156706846863462
174.2565693766521
2022-05-13 00:00:00 new price 3487.894952810833 0.0258514567090685 0.3770888082913977
103.66355273231032 103.66355273231032
152.2089180779462 64.31396526711339 0.38033003609836974
216.5228833450596
2022-02-14 00:00:00 new price 3354.858177429089 -0.0132770066385032 0.4172365124063574
114.69409053771629 114.69409053771629
66.25407060710495 111.39589317801574 0.3296266969741613
177.6499637851207
2021-11-03 00:00:00 new price 3462.0646103447593 0.0182542971602233 0.3509021489271648
96.46786074169002 96.46786074169002
115.71877311425669 53.65416276949737 0.3019405941332248
169.37293588375405
2021-08-12 00:00:00 new price 3348.72268390274 -0.0150815635580177 0.3578784650429301
98.38491378724632 98.38491378724632
53.580254140495846 104.857570237756 0.2902075798662717
158.43782437825183
2021-05-27 00

In [4]:
data['after']=post_prem
data['before']=pre_prem
data['pnl']=(data['before']-data['after'])*lot_size

In [5]:
data.to_excel("_output_eicher.xlsx")

In [52]:
#Worst case

move=data['Price_change'].abs().max()
min_drop=data['IV Drop'].min()/100

ce=black_scholes('c',fut,strike_ce,21/365, 0, current_call_iv)
pe=black_scholes('p',fut,strike_pe,21/365, 0, current_put_iv)
total_before=ce+pe
pre_prem.append(total_before)
ce_post=black_scholes('c',(1+move)*fut,strike_ce,19/365, 0, current_call_iv-min_drop)
pe_post=black_scholes('p',(1+move)*fut,strike_pe,19/365, 0, current_put_iv-min_drop)
total=ce_post+pe_post

print(total_before,total)


64.80485624661111 92.44363487395742


In [22]:
#Worst case

move=0.033
min_drop=.05

ce=black_scholes('c',fut,strike,17/365, 0, current_call_iv)
pe=black_scholes('p',fut,strike,21/365, 0, current_put_iv)
total_before=ce+pe
pre_prem.append(total_before)
ce_post=black_scholes('c',(1+move)*fut,strike,19/365, 0, current_call_iv-min_drop)
pe_post=black_scholes('p',(1+move)*fut,strike,19/365, 0, current_put_iv-min_drop)
total=ce_post+pe_post

print(total_before,total)


NameError: name 'strike' is not defined

In [12]:
#break even
points=[]
i=-0.1
while i<0.15:

    move=i
    min_drop=0.1

    ce=black_scholes('c',fut,strike_ce,17/365, 0, current_call_iv)
    pe=black_scholes('p',fut,strike_pe,17/365, 0, current_put_iv)
    print(ce,pe)
    total_before=ce+pe
    ce_post=black_scholes('c',(1+move)*fut,strike_ce,16/365, 0, current_call_iv-min_drop)
    pe_post=black_scholes('p',(1+move)*fut,strike_pe,16/365, 0, current_put_iv-min_drop)
    total=ce_post+pe_post
    points.append([i*100,total_before-total])
    i=i+0.001

17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
17.337082585263946 43.14116488506126
1

In [6]:
current_call_iv=0.104
current_put_iv=0.0875
strike_ce=42900
strike_pe=42900

strike_ce_h=43400
strike_pe_h=42400

fut=42900
lot_size=25


#grid

#break even
points=[]
i=-0.1
spots=[]
while i<0.15:

    move=i
    j=-0.1
    points_iv=[]
    ivs=[]
    while j<0.20:
        min_drop=j

        ce=black_scholes('c',fut,strike_ce,7/365, 0, current_call_iv)
        pe=black_scholes('p',fut,strike_pe,7/365, 0, current_put_iv)

        ce_h=black_scholes('c',fut,strike_ce_h,7/365, 0, current_call_iv)
        pe_h=black_scholes('p',fut,strike_pe_h,7/365, 0, current_put_iv)

        # hpe=black_scholes('p',fut,hedge_put_strike,6/365, 0, hedge_put_iv)
        total_before=ce+pe
        total_before_h=ce_h+pe_h
        ce_post=black_scholes('c',(1+move)*fut,strike_ce,1/365, 0, current_call_iv-min_drop)
        pe_post=black_scholes('p',(1+move)*fut,strike_pe,1/365, 0, current_put_iv-min_drop)

        ce_post_h=black_scholes('c',(1+move)*fut,strike_ce_h,1/365, 0, current_call_iv-min_drop)
        pe_post_h=black_scholes('p',(1+move)*fut,strike_pe_h,1/365, 0, current_put_iv-min_drop)
        # hpe_post=black_scholes('p',(1+move)*fut,hedge_put_strike,6/365, 0, hedge_put_iv-min_drop)
        total=ce_post+pe_post
        total_h=ce_post_h+pe_post_h
        points_iv.append((total-total_before)*lot_size+(total_before_h-total_h)*lot_size)
        ivs.append(int(j*100)*-1)
        j=j+0.01
    spots.append(float(i*100))
    points.append(points_iv)
    i=i+0.01/2
    print(i)

-0.095
-0.09
-0.08499999999999999
-0.07999999999999999
-0.07499999999999998
-0.06999999999999998
-0.06499999999999997
-0.05999999999999998
-0.05499999999999998
-0.04999999999999998
-0.044999999999999984
-0.03999999999999999
-0.03499999999999999
-0.02999999999999999
-0.024999999999999988
-0.019999999999999987
-0.014999999999999986
-0.009999999999999985
-0.0049999999999999845
1.5612511283791264e-17
0.005000000000000016
0.010000000000000016
0.015000000000000017
0.020000000000000018
0.02500000000000002
0.03000000000000002
0.03500000000000002
0.040000000000000015
0.04500000000000001
0.05000000000000001
0.05500000000000001
0.060000000000000005
0.065
0.07
0.07500000000000001
0.08000000000000002
0.08500000000000002
0.09000000000000002
0.09500000000000003
0.10000000000000003
0.10500000000000004
0.11000000000000004
0.11500000000000005
0.12000000000000005
0.12500000000000006
0.13000000000000006
0.13500000000000006
0.14000000000000007
0.14500000000000007
0.15000000000000008


In [6]:
#grid

#break even
points=[]
i=-0.1
spots=[]
while i<0.15:

    move=i
    j=-0.1
    points_iv=[]
    ivs=[]
    while j<0.20:
        min_drop=j

        ce=black_scholes('c',fut,strike_ce,15/365, 0, current_call_iv)
        pe=black_scholes('p',fut,strike_pe,15/365, 0, current_put_iv)
        # hpe=black_scholes('p',fut,hedge_put_strike,6/365, 0, hedge_put_iv)
        total_before=ce+pe
        ce_post=black_scholes('c',(1+move)*fut,strike_ce,14/365, 0, current_call_iv-min_drop)
        pe_post=black_scholes('p',(1+move)*fut,strike_pe,14/365, 0, current_put_iv-min_drop)
        # hpe_post=black_scholes('p',(1+move)*fut,hedge_put_strike,6/365, 0, hedge_put_iv-min_drop)
        total=ce_post+pe_post
        points_iv.append((total_before-total)*lot_size)
        ivs.append(int(j*100))
        j=j+0.01
    spots.append(float(i*100))
    points.append(points_iv)
    i=i+0.01/2
    print(i)

-0.095
-0.09
-0.08499999999999999
-0.07999999999999999
-0.07499999999999998
-0.06999999999999998
-0.06499999999999997
-0.05999999999999998
-0.05499999999999998
-0.04999999999999998
-0.044999999999999984
-0.03999999999999999
-0.03499999999999999
-0.02999999999999999
-0.024999999999999988
-0.019999999999999987
-0.014999999999999986
-0.009999999999999985
-0.0049999999999999845
1.5612511283791264e-17
0.005000000000000016
0.010000000000000016
0.015000000000000017
0.020000000000000018
0.02500000000000002
0.03000000000000002
0.03500000000000002
0.040000000000000015
0.04500000000000001
0.05000000000000001
0.05500000000000001
0.060000000000000005
0.065
0.07
0.07500000000000001
0.08000000000000002
0.08500000000000002
0.09000000000000002
0.09500000000000003
0.10000000000000003
0.10500000000000004
0.11000000000000004
0.11500000000000005
0.12000000000000005
0.12500000000000006
0.13000000000000006
0.13500000000000006
0.14000000000000007
0.14500000000000007
0.15000000000000008


In [7]:
pd.DataFrame(points,columns=ivs,index=spots).to_csv('simulation_eichermot.csv')

In [16]:
percentileofscore(data['Price_change'].abs().tolist(), 0.04)

100.0

In [24]:
lot_size

700